In [1]:
import os
from pathlib import Path

try:
  import google.colab
  IN_COLAB = True
  print("Running on a Colab Notebook environment")
except:
  IN_COLAB = False

print("os.environ.get('KAGGLE_URL_BASE') == ", os.environ.get('KAGGLE_URL_BASE'))
if 'kaggle' in os.environ.get('KAGGLE_URL_BASE','localhost'):
    print("Running on a Kaggle notebook environment")
    IN_KAGGLE = True
else:
    IN_KAGGLE = False

print(IN_COLAB)
print(IN_KAGGLE)
if IN_COLAB:
    if(os.path.isdir('input/jpx-tokyo-stock-exchange-prediction')==False):
        from google.colab import files
        files.upload()
        ! mkdir ~/.kaggle 
        ! cp kaggle.json ~/.kaggle/ 
        ! chmod 600 ~/.kaggle/kaggle.json 
        ! kaggle competitions download -c jpx-tokyo-stock-exchange-prediction
        ! mkdir input
        ! mkdir input/jpx-tokyo-stock-exchange-prediction
        ! unzip jpx-tokyo-stock-exchange-prediction.zip -d input/jpx-tokyo-stock-exchange-prediction

os.environ.get('KAGGLE_URL_BASE') ==  None
False
False


In [2]:
import warnings, gc
import numpy as np 
import pandas as pd
import matplotlib.colors
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
from datetime import datetime, timedelta
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error,mean_absolute_error
#from lightgbm import LGBMRegressor
from decimal import ROUND_HALF_UP, Decimal
warnings.filterwarnings("ignore")
import plotly.figure_factory as ff
from tqdm.auto import tqdm
import copy
import math
import random
import matplotlib.pyplot as plt

init_notebook_mode(connected=True)
temp = dict(layout=go.Layout(font=dict(family="Franklin Gothic", size=12), width=800))
colors=px.colors.qualitative.Plotly
root=Path('../input')
if IN_COLAB:
    root =Path('input')
if not IN_COLAB and not IN_KAGGLE:
    root=Path('input')
train_path=root/'jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv'


stock_list_path=root/"jpx-tokyo-stock-exchange-prediction/stock_list.csv"

train=pd.read_csv(train_path, parse_dates=['Date'])
stock_list=pd.read_csv(stock_list_path)

print("The training data begins on {} and ends on {}.\n".format(train.Date.min(),train.Date.max()))
display(train.describe().style.format('{:,.2f}'))

The training data begins on 2017-01-04 00:00:00 and ends on 2021-12-03 00:00:00.



,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,Target
count,"2,332,531.00","2,324,923.00","2,324,923.00","2,324,923.00","2,324,923.00","2,332,531.00","2,332,531.00","18,865.00","2,332,293.00"
mean,"5,894.84","2,594.51","2,626.54","2,561.23","2,594.02","691,936.56",1.00,22.02,0.00
std,"2,404.16","3,577.19","3,619.36","3,533.49","3,576.54","3,911,255.94",0.07,29.88,0.02
min,"1,301.00",14.00,15.00,13.00,14.00,0.00,0.10,0.00,-0.58
25%,"3,891.00","1,022.00","1,035.00","1,009.00","1,022.00","30,300.00",1.00,5.00,-0.01
50%,"6,238.00","1,812.00","1,834.00","1,790.00","1,811.00","107,100.00",1.00,15.00,0.00
75%,"7,965.00","3,030.00","3,070.00","2,995.00","3,030.00","402,100.00",1.00,30.00,0.01
max,"9,997.00","109,950.00","110,500.00","107,200.00","109,550.00","643,654,000.00",20.00,"1,070.00",1.12


In [3]:
def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        #print(purchase - short)
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

In [4]:


def get_label(price, code):
    """ Labelizer
    Args:
        price (pd.DataFrame): dataframe of stock_price.csv
        code (int): Local Code in the universe
    Returns:
        df (pd.DataFrame): label data
    """
    df = price.loc[price["SecuritiesCode"] == code].copy()
    df.loc[:, "label"] = df["Target"]

    return df.loc[:, ["SecuritiesCode", "label"]]



In [5]:
train.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


In [6]:
stock_list

,SecuritiesCode,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,TradeDate,Close,IssuedShares,MarketCapitalization,Universe0
0,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,1.092828e+07,3.365911e+10,True
1,1305,20211230,Daiwa ETF-TOPIX,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2097.0,3.634636e+09,7.621831e+12,False
2,1306,20211230,NEXT FUNDS TOPIX Exchange Traded Fund,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2073.5,7.917718e+09,1.641739e+13,False
3,1308,20211230,Nikko Exchange Traded Index Fund TOPIX,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2053.0,3.736943e+09,7.671945e+12,False
4,1309,20211230,NEXT FUNDS ChinaAMC SSE50 Index Exchange Trade...,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,44280.0,7.263200e+04,3.216145e+09,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4412,9994,20211230,YAMAYA CORPORATION,First Section (Domestic),Standard Market,6100,Retail Trade,14,RETAIL TRADE,7,TOPIX Small 2,20211230.0,2447.0,1.084787e+07,2.654474e+10,True
4413,9995,20211230,"GLOSEL Co.,Ltd.",First Section (Domestic),Prime Market,6050,Wholesale Trade,13,COMMERCIAL & WHOLESALE TRADE,7,TOPIX Small 2,20211230.0,410.0,2.642680e+07,1.083499e+10,False
4414,9996,20211230,"Satoh&Co.,Ltd.",JASDAQ(Standard / Domestic),Standard Market,6050,Wholesale Trade,13,COMMERCIAL & WHOLESALE TRADE,-,-,20211230.0,1488.0,9.152640e+06,1.361913e+10,False
4415,9997,20211230,"BELLUNA CO.,LTD.",First Section (Domestic),Prime Market,6100,Retail Trade,14,RETAIL TRADE,6,TOPIX Small 1,20211230.0,709.0,9.724447e+07,6.894633e+10,True


In [7]:
train_date=train.Date.unique()
returns=train.groupby('Date')['Target'].mean().mul(100).rename('Average Return')
close_avg=train.groupby('Date')['Close'].mean().rename('Closing Price')
vol_avg=train.groupby('Date')['Volume'].mean().rename('Volume')

fig = make_subplots(rows=3, cols=1, 
                    shared_xaxes=True)
for i, j in enumerate([returns, close_avg, vol_avg]):
    fig.add_trace(go.Scatter(x=train_date, y=j, mode='lines',
                             name=j.name, marker_color=colors[i]), row=i+1, col=1)
fig.update_xaxes(rangeslider_visible=False,
                 rangeselector=dict(
                     buttons=list([
                         dict(count=6, label="6m", step="month", stepmode="backward"),
                         dict(count=1, label="1y", step="year", stepmode="backward"),
                         dict(count=2, label="2y", step="year", stepmode="backward"),
                         dict(step="all")])),
                 row=1,col=1)
fig.update_layout(template=temp,title='JPX Market Average Stock Return, Closing Price, and Shares Traded', 
                  hovermode='x unified', height=700, 
                  yaxis1=dict(title='Stock Return', ticksuffix='%'), 
                  yaxis2_title='Closing Price', yaxis3_title='Shares Traded',
                  showlegend=False)
fig.show()

In [8]:

train.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


In [9]:
train_date=train.Date.unique()

open_avg=train.groupby('Date')['Open'].mean()
high_avg=train.groupby('Date')['High'].mean()
low_avg=train.groupby('Date')['Low'].mean()
close_avg=train.groupby('Date')['Close'].mean() 
buttons=[]

fig = go.Figure()

    
fig.add_trace(go.Candlestick(x=train_date, open=open_avg, high=high_avg,
                                low=low_avg, close=close_avg))

visibility=True

    
fig.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                     buttons=list([
                         dict(count=3, label="3m", step="month", stepmode="backward"),
                         dict(count=6, label="6m", step="month", stepmode="backward"),
                         dict(step="all")]), xanchor='left',yanchor='bottom', y=1.16, x=.01))
fig.update_layout(template=temp,title='Stock Price Movements by Sector', 
                  hovermode='x unified', showlegend=False, width=1000,
                  updatemenus=[dict(active=0, type="dropdown",
                                    buttons=buttons, xanchor='left',
                                    yanchor='bottom', y=1.01, x=.01)],
                  yaxis=dict(title='Stock Price'))
fig.show()

In [10]:
train

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026
...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098


In [11]:

grouped = train.groupby('Date')
box_values = []
for date, group in grouped:
    values = group['Close'].tolist()
    box_values.append(values)


In [22]:
train[['Close','Date']]

,Close,Date
0,2742.0,2017-01-04
1,571.0,2017-01-04
2,3210.0,2017-01-04
3,1550.0,2017-01-04
4,3330.0,2017-01-04
...,...,...
2332526,528.0,2021-12-03
2332527,794.0,2021-12-03
2332528,1645.0,2021-12-03
2332529,2389.0,2021-12-03


In [12]:

df = px.data.tips()

fig = px.box(df, x="day", y="total_bill", color="smoker")
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

In [13]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [14]:
train.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


In [15]:
train['Date'] = pd.to_datetime(train['Date'])

In [29]:
from datetime import date, timedelta



N = 30     # Number of boxes
startdate=date(2023,1,1)
ddates=[startdate+ timedelta(days=i) for i in range(N)]
print(ddates)


# generate an array of rainbow colors by fixing the saturation and lightness of the HSL
# representation of colour and marching around the hue.
# Plotly accepts any CSS color format, see e.g. http://www.w3schools.com/cssref/css_colors_legal.asp.
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]

# Each box is represented by a dict that contains the data, the type, and the colour.
# Use list comprehension to describe N boxes, each with a different colour and with different randomly generated data:
fig = go.Figure(data=[go.Box(
    y=3.5 * np.sin(np.pi * i/N) + i/N + (1.5 + 0.5 * np.cos(np.pi*i/N)) * np.random.rand(10),
    marker_color=c[i]
    ) for i in range(int(N))])

# format the layout
fig.update_layout(
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(zeroline=False, gridcolor='white'),
    paper_bgcolor='rgb(233,233,233)',
    plot_bgcolor='rgb(233,233,233)',
)

fig.show()

[datetime.date(2023, 1, 1), datetime.date(2023, 1, 2), datetime.date(2023, 1, 3), datetime.date(2023, 1, 4), datetime.date(2023, 1, 5), datetime.date(2023, 1, 6), datetime.date(2023, 1, 7), datetime.date(2023, 1, 8), datetime.date(2023, 1, 9), datetime.date(2023, 1, 10), datetime.date(2023, 1, 11), datetime.date(2023, 1, 12), datetime.date(2023, 1, 13), datetime.date(2023, 1, 14), datetime.date(2023, 1, 15), datetime.date(2023, 1, 16), datetime.date(2023, 1, 17), datetime.date(2023, 1, 18), datetime.date(2023, 1, 19), datetime.date(2023, 1, 20), datetime.date(2023, 1, 21), datetime.date(2023, 1, 22), datetime.date(2023, 1, 23), datetime.date(2023, 1, 24), datetime.date(2023, 1, 25), datetime.date(2023, 1, 26), datetime.date(2023, 1, 27), datetime.date(2023, 1, 28), datetime.date(2023, 1, 29), datetime.date(2023, 1, 30)]


In [30]:
from datetime import date, timedelta
import numpy as np
import plotly.graph_objs as go

N = 30     # Number of boxes
startdate=date(2023,1,1)
ddates=[(startdate+ timedelta(days=i)).strftime("%Y-%m-%d") for i in range(N)]

# generate an array of rainbow colors by fixing the saturation and lightness of the HSL
# representation of colour and marching around the hue.
# Plotly accepts any CSS color format, see e.g. http://www.w3schools.com/cssref/css_colors_legal.asp.
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]

# Each box is represented by a dict that contains the data, the type, and the colour.
# Use list comprehension to describe N boxes, each with a different colour and with different randomly generated data:
fig = go.Figure(data=[go.Box(
    y=3.5 * np.sin(np.pi * i/N) + i/N + (1.5 + 0.5 * np.cos(np.pi*i/N)) * np.random.rand(10),
    marker_color=c[i]
    ) for i in range(int(N))])

# format the layout
fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=np.arange(N),
        ticktext=ddates,
        tickangle=45
    ),
    yaxis=dict(zeroline=False, gridcolor='white'),
    paper_bgcolor='rgb(233,233,233)',
    plot_bgcolor='rgb(233,233,233)',
)

fig.show()


In [ ]:
# Group the values by day
df = pd.read_csv('time_series_data.csv', parse_dates=['timestamp'], index_col='timestamp')
df_daily = train.groupby(pd.Grouper(freq='D'))

# Create a box plot with plotly
fig = px.box(df_daily, x=df_daily.all().index.date, y='value')

# Customize the plot
fig.update_layout(
    title='Daily Boxplot',
    xaxis_title='Date',
    yaxis_title='Value',
    hovermode='closest'
)

# Show the plot
fig.show()

In [17]:
sample=pd.read_csv('input/jpx-tokyo-stock-exchange-prediction/example_test_files/sample_submission.csv')
sample

FileNotFoundError: [Errno 2] No such file or directory: 'input/jpx-tokyo-stock-exchange-prediction/example_test_files/sample_submission.csv'

For each dat, assign a number from 1 to n to stocks

In [ ]:
train_subset=train.loc[train['Date']>'2021-02-02']
train_subset

In [ ]:
# i need a function to generate portfolios:
random_sort=train_subset.SecuritiesCode.unique()
#random_sort['Rank']=np.random.rand(len(random_sort))
random_sort=pd.DataFrame(random_sort,columns=['SecuritiesCode'])
random_sort['Rank']=np.random.rand(len(random_sort))
random_sort.sort_values("Rank")
random_sort['Rank']=(random_sort['Rank'].rank(method='dense',ascending=True).astype(int))-1
random_sort

In [ ]:
SecuritiesCode=train_subset.SecuritiesCode.unique()
SecuritiesCode


In [ ]:
def generate_random_ranking_dict(securities):
    order=np.arange(len(securities))
    np.random.shuffle(order)
    randomdict={s:r for s,r in zip(securities,order)}
    #print(randomdict)
    return randomdict

def generate_random_ranking(securities):
#random_sort['Rank']=np.random.rand(len(random_sort))
    
    random_sort=pd.DataFrame(securities,columns=['SecuritiesCode'])
    random_sort['Rank']=np.random.rand(len(random_sort))
    random_sort.sort_values("Rank")
    random_sort['Rank']=(random_sort['Rank'].rank(method='dense',ascending=True).astype(int))-1
    random_sort
    return random_sort

In [ ]:
randomrank=generate_random_ranking_dict(SecuritiesCode)
train_subset['Rank']=train_subset['SecuritiesCode'].map(randomrank)
train_subset

In [ ]:
sharpes=[]

In [ ]:
def calc_spread_return_per_day(df, portfolio_size=200, toprank_weight_ratio=2):
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): spread return
    """
    assert df['Rank'].min() == 0
    assert df['Rank'].max() == len(df['Rank']) - 1
    weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
    purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
    short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
    #print(purchase - short)
    return purchase - short

In [ ]:
def generate_n_portfolios(SecuritiesCode,n):
    portfolios=[]
    for i in range(n):
        portfolios.append(generate_random_ranking_dict(SecuritiesCode))
    return portfolios

In [ ]:
portfolios=generate_n_portfolios(SecuritiesCode,10000)


In [ ]:
bufs=[]
for port in tqdm(portfolios):
    train_subset['Rank']=train_subset['SecuritiesCode'].map(port)
    buf = train_subset.groupby('Date').apply(calc_spread_return_per_day)
    bufs.append(buf)

In [ ]:
arr=np.array(bufs)
arr

In [ ]:
plt.rcParams['lines.markersize']

In [ ]:
std=np.std(arr,axis=0)

In [ ]:
with open('daily_spreads.npy','wb') as f:
    np.save(f,arr)

In [ ]:
#i want to 

In [ ]:
arr.shape

In [ ]:
n=1000

for i in tqdm(range(n)):
    randomrank=generate_random_ranking_dict(SecuritiesCode)
    train_subset['Rank']=train_subset['SecuritiesCode'].map(randomrank)
    sharpe=calc_spread_return_sharpe(train_subset)
    sharpes.append(sharpe)

In [ ]:
plt.hist(sharpes,bins=30)
plt.show()

References:

https://www.kaggle.com/code/smeitoma/jpx-competition-metric-definition

https://www.kaggle.com/code/smeitoma/submission-demo

https://www.kaggle.com/code/smeitoma/train-demo

https://www.kaggle.com/code/kellibelcher/jpx-stock-market-analysis-prediction-with-lgbm